In [ ]:
import sys
sys.path.append('../build')

from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *

Use ngbem as HOBEM solver 
=============================


As first example how it works, consider $$ \begin{cases} \Delta u &=& 0, \quad \Omega \subset \mathbb R^3\,,\\ u&=& u_0, \quad \Gamma = \partial \Omega\,.\end{cases} $$ 

Consider the following ansatz for the solution  $u\in H^1(\Omega)$ of the bvp (indirect ansatz) $$ u(x) = \int_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{1}{\| x-y\|} } \, j(y)\, \mathrm{d}\sigma_y $$ and solve for the density $j\in H^{-\frac12}(\Gamma)$ by the boundary element method.


Define the domain $\Omega \subset \mathbb R^3$ and create a mesh:

In [ ]:
#sp = Sphere( (0,0,0), 1)
#mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.3)).Curve(2)
mesh = Mesh(unit_cube.GenerateMesh(maxh=1))

Define dirichlet data:

In [ ]:
u0 = 1/ sqrt( (x+1)**2 + (y+1)**2 + (z+1)**2 )

Create test and trial function finite element spaces for $H^{-\frac12}(\Gamma)$ according to the given mesh:  

In [ ]:
fesL2 = SurfaceL2(mesh, order=3, dual_mapping=True)
u,v = fesL2.TnT()

Assemble single layer potential operator matrix and the right hand side: 

In [ ]:
a = BilinearForm(fesL2)
SingleLayerPotentialOperator(a, intorder=10)
a.Assemble();
f = LinearForm (u0*v.Trace()*ds(bonus_intorder=3)).Assemble()

Solve the linear system of equations and have a look at the approximation of density $j$:

In [ ]:
gfv = GridFunction(fesL2)
inv = a.mat.Inverse()
gfv.vec.data = inv * f.vec
Draw (gfv);

Consider alternative the following ansatz for the solution $u\in H^1(\Omega)$ of the above bvp (direct ansatz) $$ u(x) = \int_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{1}{\| x-y\|} } \, u_1(y)\, \mathrm{d}\sigma_y - \int_\Gamma n(y)\cdot \displaystyle{\frac{1}{4\,\pi}\, \frac{x-y}{\| x-y\|^3} } \, u_0(y)\, \mathrm{d}\sigma_y  $$ and solve for the Neumann data $u_1 \in H^{-\frac12}(\Gamma)$ by the boundary element method. 

We assemble the double layer potential operator with finite element spaces for $H^{\frac12}(\Gamma)$ and $H^{-\frac12}(\Gamma)$: 

In [ ]:
fesH1 = H1(mesh, order=3)
uH1,vH1 = fesH1.TnT()
b = BilinearForm(trialspace=fesH1, testspace=fesL2)
DoubleLayerPotentialOperator(b, intorder=12)
b += - 0.5 * uH1 * v.Trace() * ds(bonus_intorder=3)
b.Assemble();

We interpolate the given dirichlet trace in finite element space for $H^{-\frac12}(\Gamma)$, compute the rhs and solve the linear system:

In [ ]:
gfdir = GridFunction(fesH1)
gfdir.Interpolate (u0)
gfv.vec.data = inv @ b.mat * gfdir.vec

In [ ]:
Draw (-gfv, mesh, draw_vol=False);

The solution of the direct ansatz are Neumann data of $u$. Let's look at the exact Neumann trace: 

In [ ]:
gradu0 = CF( (u0.Diff(x), u0.Diff(y), u0.Diff(z)) )
n = specialcf.normal(3)
u1 = gradu0*n
Draw (u1, mesh, draw_vol=False);

You can find details on the implementation of isoparametric HOBEM for elliptic and Maxwell problems in the following work:


[High Order Boundary Elment Methods (2011)](https://publikationen.sulb.uni-saarland.de/bitstream/20.500.11880/26312/1/thesis_weggler_final_6.1.12.pdf)

